# Proyecto Parcial



In [1]:
%%info

UsageError: Cell magic `%%info` not found.


In [2]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

UsageError: Cell magic `%%configure` not found.


In [3]:
# Importar librerías
from pyspark.sql import SparkSession

# Importar funciones únicas
from pyspark.sql.functions import lit, col

# Importar funciones con un alias, que pudieran 
# tener conflicto con las librerías base de Python, 
# como sum, avg, round, ...
import pyspark.sql.functions as F
from pyspark.sql.types import (
    StructType, StringType, TimestampType, 
    StringType, StringType, FloatType, StructField, 
    LongType, IntegerType)

from pyspark.sql.functions import regexp_replace


raw_def_columns = StructType([
    StructField("producto", StringType(), True),
    StructField("presentacion", StringType(), True),
    StructField("marca", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("catalogo", StringType(), True),
    StructField("precio", FloatType(), True),
    StructField("fecha", StringType(), True),
    StructField("tienda", StringType(), True),
    StructField("tipo_tienda", LongType(), True),
    StructField("sucursal", StringType(), True),
    StructField("direccion", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("ciudad", StringType(), True),
    StructField("latitud", IntegerType(),True),
    StructField("longitud", IntegerType(), True)
]) 



Recuerda que vamos a trabajar con spark dataframes, por lo tanto tienes que abrir un SparkSession

In [7]:
spark = SparkSession.builder.appName("CleanCompleteFile").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/05/03 14:27:17 WARN Utils: Your hostname, ThomasLOQ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/03 14:27:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/03 14:27:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark._sc.install_pypi_package("pandas")

AttributeError: 'SparkContext' object has no attribute 'install_pypi_package'

In [5]:
# bucket
NAME = 'thmrudolf' ##CAMBIAR AQUÍ SU NOMBRE.
BUCKET = f"s3://itam-analytics-{NAME}"
FOLDER = 'profeco'


# type of catalog
CATALOG_TYPE = 'basicos'

In [10]:
#Descarga de datos de S3
#raw_df = spark.read.csv(f"{BUCKET}/{FOLDER}/raw_descomprimidos/*/*.csv", 
raw_df = spark.read.csv(f"{BUCKET}/{FOLDER}/raw_descomprimidos/2024/*.csv", 
                        header=True, 
                        inferSchema=True, 
                       schema=raw_def_columns)


25/05/03 14:27:41 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3://itam-analytics-thmrudolf/profeco/raw_descomprimidos/2024/*.csv.
org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.load

Py4JJavaError: An error occurred while calling o34.csv.
: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3443)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(DataSource.scala:722)
	at org.apache.spark.sql.execution.datasources.DataSource.checkAndGlobPathIfNecessary(Data

In [11]:
raw_df.printSchema()


NameError: name 'raw_df' is not defined

### Limpieza de datos

In [ ]:
columnas = [
    "producto", "presentacion", "marca", "tipo", "catalogo",
    "precio", "fecha", "tienda", "tipo_tienda", "sucursal",
    "direccion", "estado", "ciudad", "latitud", "longitud"
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for column in raw_df.columns:
    raw_df = raw_df.withColumn(column, regexp_replace(col(column), '""', "'"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.functions import expr

# Count occurrences of `""` before replacement
counts = [expr(f"sum(length({c}) - length(regexp_replace({c}, '\"\"', '')))").alias(f"{c}_count") for c in raw_df.columns]

# Show count for each column
raw_df.select(*counts).show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------+-----------+----------+--------------+------------+-----------+------------+-----------------+--------------+---------------+------------+------------+-------------+--------------+
|producto_count|presentacion_count|marca_count|tipo_count|catalogo_count|precio_count|fecha_count|tienda_count|tipo_tienda_count|sucursal_count|direccion_count|estado_count|ciudad_count|latitud_count|longitud_count|
+--------------+------------------+-----------+----------+--------------+------------+-----------+------------+-----------------+--------------+---------------+------------+------------+-------------+--------------+
|             0|                 0|          0|         0|             0|           0|          0|           0|             NULL|             0|              0|           0|           0|         NULL|          NULL|
+--------------+------------------+-----------+----------+--------------+------------+-----------+------------+-----------------+-------

In [ ]:
#Cambiamos tipo a variable fecha
# Convertimos columnas a tipo correcto
from pyspark.sql.functions import to_date, col
from pyspark.sql.functions import year, month, dayofmonth


raw_df = raw_df.withColumn("fecha", to_date(col("fecha"), "yyyy-MM-dd"))
raw_df = raw_df \
    .withColumn("anio", year(col("fecha"))) \
    .withColumn("mes", month(col("fecha"))) \
    .withColumn("dia", dayofmonth(col("fecha")))

# Revisamos que esté todo en orden
print(raw_df.dtypes)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('producto', 'string'), ('presentacion', 'string'), ('marca', 'string'), ('tipo', 'string'), ('catalogo', 'string'), ('precio', 'string'), ('fecha', 'date'), ('tienda', 'string'), ('tipo_tienda', 'string'), ('sucursal', 'string'), ('direccion', 'string'), ('estado', 'string'), ('ciudad', 'string'), ('latitud', 'string'), ('longitud', 'string'), ('anio', 'int'), ('mes', 'int'), ('dia', 'int')]

In [ ]:
#Eliminamos las columnas dirección, latitud y longitud
col_to_drop =['direccion', 'latitud', 'longitud', 'presentacion', 'tienda', 'tipo_tienda','sucursal']
raw_df = raw_df.drop(*col_to_drop)
raw_df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- producto: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fecha: date (nullable = true)
 |-- estado: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)

In [ ]:
#Convertimos el contenido a minúsculas for col in raw_df.columns: 

from pyspark.sql.functions import lower, col

# Convert all string columns to lowercase
for column in raw_df.columns:
    raw_df = raw_df.withColumn(column, lower(col(column)))

raw_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+-----------------+------+----------+--------------+--------------+----+---+---+
|            producto|               marca|                tipo|         catalogo|precio|     fecha|        estado|        ciudad|anio|mes|dia|
+--------------------+--------------------+--------------------+-----------------+------+----------+--------------+--------------+----+---+---+
|     barra de sonido|                  lg|aparatos electron...|electrodomesticos|4299.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
|     barra de sonido|                sony|aparatos electron...|electrodomesticos|4499.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
|     barra de sonido|                sony|aparatos electron...|electrodomesticos|4899.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
|           batidoras|               oster| aparatos electricos|electrodomesticos| 669.0|2024-01-02|aguascalientes|aguascalientes|2024| 

In [ ]:
spark._sc.install_pypi_package("Unidecode")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import unidecode

# List of string columns to normalize
string_columns = [field.name for field in raw_df.schema.fields if field.dataType.simpleString() == "string"]

# Define UDF to remove accents
def remove_accents(text):
    if text:
        return unidecode.unidecode(text)
    return text

remove_accents_udf = udf(remove_accents, StringType())

# Apply normalization
for column in string_columns:
    raw_df = raw_df.withColumn(column, remove_accents_udf(col(column)))

raw_df.show(5)  # Show first 5 rows

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+--------------------+-----------------+------+----------+--------------+--------------+----+---+---+
|       producto|marca|                tipo|         catalogo|precio|     fecha|        estado|        ciudad|anio|mes|dia|
+---------------+-----+--------------------+-----------------+------+----------+--------------+--------------+----+---+---+
|barra de sonido|   lg|aparatos electron...|electrodomesticos|4299.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
|barra de sonido| sony|aparatos electron...|electrodomesticos|4499.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
|barra de sonido| sony|aparatos electron...|electrodomesticos|4899.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
|      batidoras|oster| aparatos electricos|electrodomesticos| 669.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
|      batidoras|oster| aparatos electricos|electrodomesticos|2699.0|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2|
+-------

In [ ]:
#Subimos los datos a S3 en formato parquet
raw_df.repartition("catalogo") \
  .write.mode("overwrite") \
  .partitionBy("catalogo") \
  .option("compression", "snappy") \
  .parquet(f"{BUCKET}/{FOLDER}/parquet")
# Comprobamos que se haya subido correctamente
df = spark.read.parquet(f"{BUCKET}/{FOLDER}/parquet")
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+--------------------+------+----------+--------------+--------------+----+---+---+--------+
|producto|     marca|                tipo|precio|     fecha|        estado|        ciudad|anio|mes|dia|catalogo|
+--------+----------+--------------------+------+----------+--------------+--------------+----+---+---+--------+
|   arroz|precissimo|arroz y cereales ...|  14.2|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2| basicos|
|    atun|   dolores|pescados y marisc...|  23.1|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2| basicos|
|    atun| el dorado|pescados y marisc...|  13.9|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2| basicos|
|    atun| el dorado|pescados y marisc...|  13.9|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2| basicos|
|    atun|    herdez|pescados y marisc...|  22.9|2024-01-02|aguascalientes|aguascalientes|2024|  1|  2| basicos|
|    atun|    herdez|pescados y marisc...|  23.5|2024-01-02|aguascalientes|aguascalientes|2024| 

## Parte A

Contesta las siguientes preguntas utilizando PySpark. Realiza el siguiente análisis (por año) y sobre todos los catálogos.

### ¿Cuántos catálogos diferentes tenemos?

In [ ]:
from pyspark.sql.functions import col

# Contar los catálogos distintos en el DataFrame
num_catalogos = raw_df.select(col("catalogo")).distinct().count()

print(f"Total de catálogos diferentes: {num_catalogos}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total de cat?logos diferentes: 10

### ¿Cuáles son los 20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.

In [ ]:
from pyspark.sql.functions import col

# Contar observaciones por catálogo y ordenar en orden descendente
top_catalogos = raw_df.groupBy("catalogo") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(20)

# Mostrar los resultados
top_catalogos.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------+
|           catalogo|  count|
+-------------------+-------+
|            basicos|1986577|
|       medicamentos| 798628|
|  electrodomesticos| 267830|
| frutas y legumbres| 211838|
|              pacic| 155798|
|           mercados|  76886|
|   utiles escolares|  62102|
|pescados y mariscos|  45966|
|           juguetes|  19114|
|          navidenos|    202|
+-------------------+-------+

In [ ]:
top_catalogos.write.mode("overwrite") \
    .option("compression", "snappy") \
    .parquet(f"s3a://{BUCKET}/{FOLDER}/top_catalogos")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o562.parquet.
: org.apache.hadoop.fs.s3a.AWSBadRequestException: getFileStatus on s3a://s3:/itam-analytics-thmrudolf/profeco/top_catalogos: software.amazon.awssdk.services.s3.model.S3Exception: Bad Request (Service: S3, Status Code: 400, Request ID: VX45NK25BK03AQ0X, Extended Request ID: Jb43GtvmyIpSeVWVIWfEP0h3WgvaaiMGVq/ep4URDBgR3b3vz7ZXEfdDmGssSzq+b8HMvl0ZfMk=):null: Bad Request (Service: S3, Status Code: 400, Request ID: VX45NK25BK03AQ0X, Extended Request ID: Jb43GtvmyIpSeVWVIWfEP0h3WgvaaiMGVq/ep4URDBgR3b3vz7ZXEfdDmGssSzq+b8HMvl0ZfMk=)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:269)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:162)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:4317)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:4230)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$exists$33(S3AFileSystem

### ¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

1. Obtener los estados presentes en el dataset

In [ ]:
from pyspark.sql.functions import col

# Obtener lista de estados únicos en los datos
estados_en_datos = raw_df.select(col("estado")).distinct()
estados_en_datos.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|              estado|
+--------------------+
|          nuevo leon|
|            campeche|
|             morelos|
|          guanajuato|
|              oaxaca|
|            guerrero|
|           queretaro|
|    estado de mexico|
|              puebla|
|             durango|
|             jalisco|
|      aguascalientes|
|coahuila de zaragoza|
| baja california sur|
|           chihuahua|
|     baja california|
|    ciudad de mexico|
|             hidalgo|
| michoacan de ocampo|
|             chiapas|
+--------------------+
only showing top 20 rows

2. Comparar con la lista completa de estados

In [ ]:
estados_mexico = [
    "aguascalientes", "baja california", "baja california sur", "campeche", "coahuila",
    "colima", "chiapas", "chihuahua", "durango", "estado de mexico", "guanajuato", "guerrero", "hidalgo",
    "jalisco", "ciudad de mexico", "michoacan", "morelos", "nayarit", "nuevo leon", "oaxaca",
    "puebla", "queretaro", "quintana roo", "san luis potosi", "sinaloa", "sonora",
    "tabasco", "tamaulipas", "tlaxcala", "veracruz", "yucatan", "zacatecas"
]

# Convertir PySpark DataFrame a lista para comparación
estados_en_datos_lista = [row.estado for row in estados_en_datos.collect()]

# Encontrar estados faltantes
estados_faltantes = list(set(estados_mexico) - set(estados_en_datos_lista))

print(f"Estados faltantes en los datos: {estados_faltantes}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Estados faltantes en los datos: ['michoacan', 'coahuila', 'colima', 'nayarit']

### De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [ ]:
from pyspark.sql.functions import countDistinct

# Contar catálogos distintos por estado y año
catalogos_por_estado_anio = raw_df.groupBy("estado", "anio") \
    .agg(countDistinct("catalogo").alias("num_catalogos"))

# Mostrar resultados
catalogos_por_estado_anio.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------+
|              estado|anio|num_catalogos|
+--------------------+----+-------------+
|            guerrero|2024|            9|
|coahuila de zaragoza|2024|           10|
|            tlaxcala|2024|            9|
|          nuevo leon|2024|           10|
|           queretaro|2024|           10|
|             chiapas|2024|            8|
|              sonora|2024|            9|
|            veracruz|2024|           10|
| baja california sur|2024|           10|
|          guanajuato|2024|            9|
+--------------------+----+-------------+
only showing top 10 rows

Análisis: ¿Ha aumentado el número de catálogos con el tiempo?

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

# Definir ventana por estado para comparar años anteriores
window_spec = Window.partitionBy("estado").orderBy("anio")

# Calcular la diferencia de catálogos con el año anterior
catalogos_por_estado_anio = catalogos_por_estado_anio.withColumn(
    "diferencia_anual",
    col("num_catalogos") - lag("num_catalogos", 1).over(window_spec)
)

catalogos_por_estado_anio.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+-------------+----------------+
|             estado|anio|num_catalogos|diferencia_anual|
+-------------------+----+-------------+----------------+
|             mz. 54|2024|            1|            NULL|
|     aguascalientes|2024|            9|            NULL|
|    baja california|2024|           10|            NULL|
|baja california sur|2024|           10|            NULL|
|           campeche|2024|            9|            NULL|
+-------------------+----+-------------+----------------+
only showing top 5 rows

In [ ]:
df_filtered = raw_df.filter(raw_df["estado"] == "mz. 54")
df_filtered.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+----+--------+------+-----+------+------+----+---+---+
|producto|marca|tipo|catalogo|precio|fecha|estado|ciudad|anio|mes|dia|
+--------+-----+----+--------+------+-----+------+------+----+---+---+
+--------+-----+----+--------+------+-----+------+------+----+---+---+

In [ ]:
print(df_filtered.collect()) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]